# 横向联邦统计任务示例

这是一个使用Delta框架编写的横向联邦统计的任务示例。

数据是分布在各个节点上的csv文件，文件内容是公司内所有员工的工资。任务是计算这些公司的平均工资。

本示例可以直接在Deltaboard中执行并查看结果。<span style="color:#FF8F8F;font-weight:bold">在点击执行之前，需要修改一下个人的Deltaboard API的地址，具体请看下面第4节的说明。</span>


## 1. 引入需要的包

我们需要```delta-task```的包中，引入Delta框架的内容，包括```DeltaNode```节点，用于调用API发送任务，以及用于横向联邦统计任务```HorizontalAnalytics```。

In [ ]:
from delta import pandas as pd
from delta.task import HorizontalAnalytics
from delta.delta_node import DeltaNode
import delta.dataset

from typing import Dict

## 2. 定义隐私计算任务

然后可以开始定义我们的横向统计任务了，用横向联邦统计的方式，在多节点上统计我们想要的数据。

在定义横向联邦统计任务时，有几部分内容是需要用户自己定义的：

* ***任务配置***: 我们需要在 ```super().__init__()``` 方法中对任务进行配置。 这些配置项包括任务名称（```name```），所需的最少客户端数（```min_clients```），最大客户端数（```max_clients```），等待超时时间（```wait_timeout```，用来控制一轮计算的超时时间），以及连接超时时间（```connection_timeout```，用来控制流程中每个阶段的超时时间）。
* ***数据集***: 我们需要在```dataset```方法中定义任务所需要的数据集。 该方法返回一个字典，键是数据集的名称，需要与execute方法的参数名对应；对应的值是```delta.dataset.DataFrame```实例， 其参数```dataset```代表所需数据集的名称。关于数据集格式的具体细节，请参考[这篇文章](https://docs.deltampc.com/network-deployment/prepare-data)。
* ***统计的逻辑***: 我们需要在execute方法中实现所有统计的逻辑。execute方法的输入需要与```dataset```方法的返回值对应，即一个输入形参，对应```dataset```返回的字典中的一项。目前参数类型只支持```delta.pandas.DataFrame```这一种。```delta.pandas.DataFrame```的方法类似```pandas.DataFrame```，目前支持```+,-,*,/,//,%```等操作符，以及```all, any, count, sum, mean, std, var, sem```这些方法。


In [ ]:
class WageAvg(HorizontalAnalytics):
    def __init__(self) -> None:
        super().__init__(
            name="wage_avg",  # 任务名称，用于在Deltaboard中的展示
            min_clients=2,  # 算法所需的最少客户端数，至少为2
            max_clients=3,  # 算法所支持的最大客户端数，必须大雨等于min_clients
            wait_timeout=5,  # 等待超时时间，用来控制一轮计算的超时时间
            connection_timeout=5,  # 连接超时时间，用来控制流程中每个阶段的超时时间
        )

    def dataset(self) -> Dict[str, delta.dataset.DataFrame]:
        """
        定义任务所需的数据。
        return: 字典，键是数据的名字，需要与execute方法中的参数名称对应；值是一个delta.dataset.DataFrame实例。
        """
        return {
            "wages": delta.dataset.DataFrame("wages.csv")
        }

    def execute(self, wages: pd.DataFrame):
        """
        实现具体的统计逻辑。
        输入与dataset方法的返回值对应
        """
        return wages.mean()


## 4. 指定执行任务用的Delta Node的API

定义好了任务，我们就可以开始准备在Delta Node上执行任务了。

Delta Task框架可以直接调用Delta Node API发送任务到Delta Node开始执行，只要在任务执行时指定Delta Node的API地址即可。

Deltaboard提供了对于Delta Node的API的封装，为每个用户提供了一个独立的API地址，支持多人同时使用同一个Delta Node，并且能够在Deltaboard中管理自己提交的任务。
在这里，我们使用Deltaboard提供的API来执行任务。如果用户自己搭建了Delta Node，也可以直接使用Delta Node的API。

在左侧导航栏中进入“个人中心”，在Deltaboard API中，复制自己的API地址，并粘贴到下面的代码中：

In [ ]:
DELTA_NODE_API = "http://127.0.0.1:6704"

## 5. 执行隐私计算任务

接下来我们可以开始运行这个模型了：

In [ ]:
task = WageAvg().build()
delta_node = DeltaNode(DELTA_NODE_API)
delta_node.create_task(task)

## 6. 查看执行状态
点击执行后，可以从输出的日志看出，任务已经提交到了Delta Node的节点上。

接下来，可以从左侧的导航栏中，前往“任务列表”，找到刚刚提交的任务，点击进去查看具体的执行日志了。